In [1]:
import pandas as pd

train = pd.read_csv('cola_train.csv')
test = pd.read_csv('cola_test.csv')
vali = pd.read_csv('cola_validation.csv')
print(train.head())
print(test.head())

                                            sentence  label  idx
0  Our friends won't buy this analysis, let alone...      1    0
1  One more pseudo generalization and I'm giving up.      1    1
2   One more pseudo generalization or I'm giving up.      1    2
3     The more we study verbs, the crazier they get.      1    3
4          Day by day the facts are getting murkier.      1    4
                                            sentence  label  idx
0                      Bill whistled past the house.     -1    0
1              The car honked its way down the road.     -1    1
2                    Bill pushed Harry off the sofa.     -1    2
3               the kittens yawned awake and played.     -1    3
4  I demand that the more John eats, the more he ...     -1    4


In [2]:
train_input=train['sentence']
train_label=train['label']
test_input=test['sentence']
test_label=test['label']
vali_input=vali['sentence']
vali_label=vali['label']

In [3]:
train_input

0       Our friends won't buy this analysis, let alone...
1       One more pseudo generalization and I'm giving up.
2        One more pseudo generalization or I'm giving up.
3          The more we study verbs, the crazier they get.
4               Day by day the facts are getting murkier.
                              ...                        
8546                     Poseidon appears to own a dragon
8547                       Digitize is my happiest memory
8548                       It is easy to slay the Gorgon.
8549         I had the strangest feeling that I knew you.
8550                  What all did you get for Christmas?
Name: sentence, Length: 8551, dtype: object

In [23]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Hugging Face의 Dataset 객체로 변환
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)
vali_dataset = Dataset.from_pandas(vali)
def preprocess_function(examples):
    return tokenizer(examples['sentence'], padding='max_length', truncation=True, max_length=30)

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)
vali_dataset = vali_dataset.map(preprocess_function, batched=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

In [20]:
train_dataset

Dataset({
    features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 8551
})

In [21]:
max([len(train_dataset[i]['input_ids']) for i in range(8551)])

30

In [24]:
# Trainer 준비
training_args = TrainingArguments(
    output_dir='./results',           # 출력 디렉토리
    num_train_epochs=3,               # 에포크 수
    per_device_train_batch_size=8,    # 훈련 배치 크기
    per_device_eval_batch_size=8,     # 평가 배치 크기
    warmup_steps=500,                 # 워밍업 스텝
    weight_decay=0.01,                # 가중치 감쇠
    logging_dir='./logs',             # 로깅 디렉토리
    logging_steps=10,
)

trainer = Trainer(
    model=model,                       # 파인튜닝할 모델
    args=training_args,                # 훈련 인자
    train_dataset=train_dataset,       # 훈련 데이터셋
    eval_dataset=vali_dataset          # 평가 데이터셋
)

# 모델 훈련
trainer.train()

# 평가
results = trainer.evaluate()
print(results)

Step,Training Loss
10,1.109900
20,1.036500
30,1.013700
40,0.969200
50,0.918600
60,0.891200
70,0.864500
80,0.687600
90,0.698000
100,0.659100


KeyboardInterrupt: 